# Avg_profiles_Mix Function

This function will compute the vertical profiles and the integral of u(z)*rho*C(z)

This function is called in the following notebooks:

    ...
    ...
    ...

In [1]:
import numpy as numpy 
#can't use np since that is used as a variable for length within the function

from mpmath import *
mp.dps = 15; mp.pretty = True
# allows us to use the exponential integral function ein()

In [5]:
def avg_profiles_mix(h,settl_vel,k,friction_coeff,rhoC_avg,
                     u_guess,beta,h0,b,u_coeff,u_rel0,rhos,rhog):
    
    # The input here are the depth-averaged velocity and the depth-averaged
    # values of rho*C(z). We compute the vertical profiles and the integral of 
    # u(z)*rho*C(z), and we compare this value with the desired one to update
    # the average velocity.

    # Shear velocity computed from u_guess
    u_star = u_guess * numpy.sqrt(friction_coeff) 

    # Rouse numbers for the particle classes
    Pn = settl_vel / ( k * u_star ) 

    # number of particle classes
    np = len(settl_vel) 

    # array for depth-averaged value of rho*u(z)*C(z)
    rho_u_C = numpy.zeros(1,np) 

    rhoC_int = numpy.zeros(1,np) 

    for i in range(0,np-1):

        a = -(0.6*Pn[i]*beta)/h  

        int_ = ( ( numpy.exp(a*h0) -1.0 )/a + numpy.exp(a*h0)*(h-h0) )/h 
        # Had to rename variable 'int' to 'int_' since int appears as a built in function

        C_rel_max = 1.0/int_ 

        # relative concentration C_rel at depth h0 (from the bottom)
        # C_rel is defined as C(z)/C_avg
        C_rel0 = C_rel_max * numpy.exp(a*h0) 

        # depth-averaged value of rho*C(z)
        # this results from the sum of the log region with thickness h0 and the 
        # constant region with thickness h-h0
        rhoC_int[i] = rhoC_avg[i] * ( C_rel_max * ( numpy.exp(a*h0) - 1.0 ) / a + (h-h0)*C_rel0 ) / h   

        # we compute the integral in the log region of u_tilde(z)*C_rel(z), where 
        # u_tilde is defined as u(z)/(u_guess*u_coeff*sqrt(friction_coeff)) 
        y = h0 
        int_h0 = ( numpy.exp(a*y)*numpy.log(b*y+1.0) + numpy.exp(-a/b) * ein( -a*(y+1.0/b) ) ) / a 
        y = 0 
        int_0 = ( numpy.exp(a*y)*numpy.log(b*y+1.0) + numpy.exp(-a/b) * ein( -a*(y+1.0/b) ) ) / a 

        # integral of u_rel(z)*C_rel(z) in the log region (0<=z<=h0)
        # here u_rel(z) = u(z)/u_guess 
        int_def = u_coeff * numpy.sqrt(friction_coeff) / k * C_rel_max * (int_h0-int_0) 

        # we add the contribution of the integral of the constant region, we
        # average by dividing by h and we multiply by the density of solid and
        # average concentration and by u_guess.
        rho_u_C[i] = rhoC_avg[i] * u_guess * ( int_def + ( h - h0 ) * C_rel0 * u_rel0 ) / h 

    # we add the contribution of the gas phase to the depth-averaged mixture 
    # density. This value should be equal to that used to compute the input
    # values rhoC_avg.
    rhom_avg = rhog + sum((rhos-rhog)/rhos*rhoC_int) 

    # we add the contribution of the gas phase to the mixture depth-averaged 
    # momentum
    uRho_avg_new = ( u_guess*rhog + sum((rhos-rhog)/rhos*rho_u_C) ) 
